In [1]:
# Connect To PostgreSQL Database Server using the psycopg database adapter
#!pip install psycopg2

In [2]:
# libraries
import psycopg2
import pandas as pd

In [3]:
# Connect to the PostgreSQL database using the psycopg2
# https://www.postgresqltutorial.com/postgresql-python/connect/
# https://stackoverflow.com/questions/62555244/unable-to-connect-to-postgres-sever-from-google-colab-using-psycopg2
conn = psycopg2.connect(
    host="localhost",
    database="topups1",
    user="postgres",
    password="trinhien")

In [4]:
curr = conn.cursor()
print('PostgreSQL database version:')  # version of PostgreSQL
curr.execute('SELECT version()')

db_version = curr.fetchone()
print(db_version)

curr.execute("ROLLBACK")
conn.commit()
# fix DatabaseError: current transaction is aborted, commands ignored until end of transaction block

PostgreSQL database version:
('PostgreSQL 14.2, compiled by Visual C++ build 1914, 64-bit',)


In [5]:
# create a foreign server named topups
sql1 = '''CREATE SERVER topups_v2 FOREIGN DATA WRAPPER file_fdw;'''
curr.execute(sql1)

In [6]:
# create a foreign table
sql2 = '''CREATE FOREIGN TABLE foreign_topups_v2(
    seq varchar(20),
    id_user varchar(20),
    topup_date varchar(20),
    topup_value varchar(20))
    SERVER topups_v2
    OPTIONS (format 'csv', header 'true', filename 'C:/Users/hient/OneDrive - National University of Ireland, Galway/Personal documents/Job/Sonra/topups.tsv', delimiter E'\t');
    '''
curr.execute(sql2)

In [7]:
# load as topups table 
sql3 = '''CREATE TABLE topups_v2 AS SELECT * FROM foreign_topups_v2;'''
curr.execute(sql3)

In [8]:
# view topups table 
sql4 = '''SELECT * FROM topups_v2;'''
curr.execute(sql4)

In [9]:
# change data type for each column 
sql5 = '''ALTER TABLE topups_v2
    ALTER COLUMN seq TYPE INTEGER USING (seq::INTEGER),
    ALTER COLUMN id_user TYPE INTEGER USING (id_user::INTEGER),
    ALTER COLUMN topup_date TYPE DATE USING to_date(topup_date, 'YYYY-MM-DD'),
    ALTER COLUMN topup_value TYPE INTEGER USING (topup_value::INTEGER);'''
curr.execute(sql5)

In [10]:
# task 2
sql6 = '''SELECT id_user, SUM(topup_value) AS total_topup FROM topups_v2 
                WHERE id_user IN (
                SELECT id_user FROM topups WHERE topup_value = 15 GROUP BY id_user HAVING COUNT(id_user) >= 1 
                )
                GROUP BY id_user;'''
curr.execute(sql6)
for row in curr.fetchall():
    print(row)

(4, 225)
(3, 215)
(1, 185)
(5, 235)
(2, 95)


In [11]:
# task 3
sql7 = '''SELECT * FROM (
            SELECT *, 
                    ROW_NUMBER() OVER (PARTITION BY id_user ORDER BY topup_date DESC, topup_value DESC) AS date_rank           
                    FROM topups_v2
            ) RANKS WHERE date_rank <= 5;'''
curr.execute(sql7)
for row in curr.fetchall():
    print(row)

(65, 1, datetime.date(2017, 6, 20), 15, 1)
(54, 1, datetime.date(2017, 5, 22), 10, 2)
(47, 1, datetime.date(2017, 4, 17), 20, 3)
(42, 1, datetime.date(2017, 3, 20), 20, 4)
(34, 1, datetime.date(2017, 2, 20), 15, 5)
(66, 2, datetime.date(2017, 6, 20), 5, 1)
(59, 2, datetime.date(2017, 6, 5), 10, 2)
(48, 2, datetime.date(2017, 4, 22), 5, 3)
(44, 2, datetime.date(2017, 3, 30), 10, 4)
(39, 2, datetime.date(2017, 3, 15), 15, 5)
(68, 3, datetime.date(2017, 6, 28), 15, 1)
(55, 3, datetime.date(2017, 5, 25), 5, 2)
(53, 3, datetime.date(2017, 5, 11), 25, 3)
(52, 3, datetime.date(2017, 5, 11), 15, 4)
(45, 3, datetime.date(2017, 4, 5), 25, 5)
(64, 4, datetime.date(2017, 6, 17), 25, 1)
(61, 4, datetime.date(2017, 6, 7), 10, 2)
(60, 4, datetime.date(2017, 6, 5), 5, 3)
(56, 4, datetime.date(2017, 5, 30), 25, 4)
(49, 4, datetime.date(2017, 4, 22), 10, 5)
(67, 5, datetime.date(2017, 6, 26), 5, 1)
(63, 5, datetime.date(2017, 6, 15), 20, 2)
(62, 5, datetime.date(2017, 6, 12), 5, 3)
(58, 5, datetime.date

In [12]:
# task 4
sql8 = '''SELECT * FROM (
            SELECT *, RANK() over (PARTITION BY id_user ORDER BY topup_value DESC) AS rrank           
            FROM topups
            ) RANKS WHERE rrank <= 5;'''
curr.execute(sql8)
for row in curr.fetchall():
    print(row)

(1, 1, datetime.date(2016, 11, 7), 25, 1)
(29, 1, datetime.date(2017, 1, 28), 25, 1)
(21, 1, datetime.date(2017, 1, 3), 25, 1)
(42, 1, datetime.date(2017, 3, 20), 20, 4)
(7, 1, datetime.date(2016, 12, 5), 20, 4)
(47, 1, datetime.date(2017, 4, 17), 20, 4)
(23, 2, datetime.date(2017, 1, 15), 20, 1)
(39, 2, datetime.date(2017, 3, 15), 15, 2)
(59, 2, datetime.date(2017, 6, 5), 10, 3)
(31, 2, datetime.date(2017, 2, 10), 10, 3)
(44, 2, datetime.date(2017, 3, 30), 10, 3)
(32, 2, datetime.date(2017, 2, 10), 10, 3)
(30, 3, datetime.date(2017, 2, 1), 25, 1)
(14, 3, datetime.date(2016, 12, 19), 25, 1)
(45, 3, datetime.date(2017, 4, 5), 25, 1)
(53, 3, datetime.date(2017, 5, 11), 25, 1)
(40, 3, datetime.date(2017, 3, 15), 20, 5)
(64, 4, datetime.date(2017, 6, 17), 25, 1)
(56, 4, datetime.date(2017, 5, 30), 25, 1)
(24, 4, datetime.date(2017, 1, 19), 25, 1)
(41, 4, datetime.date(2017, 3, 15), 25, 1)
(17, 4, datetime.date(2016, 12, 31), 20, 5)
(16, 4, datetime.date(2016, 12, 20), 20, 5)
(33, 4, dateti

In [13]:
# task 5
# tem_task5: CTE (temporary)
sql9 = '''CREATE TABLE task5_v2 AS
WITH tem_task5_v2 AS (SELECT t1.id_user, t1.topup_date, t1.topup_value, 
		t1.prv_topup_dt, t1.promo_ind, t3.previous_qual_topup_dt,
		t1."topup_date"::date - t1."prv_topup_dt"::date AS days_since,
		ROUND (t1.topup_value * 1. / t2.min_topup, 1) AS to_1st_ratio FROM (
	SELECT *,
        LEAD(topup_date) OVER (PARTITION BY id_user ORDER BY topup_date DESC) AS prv_topup_dt,
		CASE WHEN topup_value >= 20 THEN 'Y' ELSE 'N' END AS promo_ind
		FROM topups_v2) t1
		
LEFT JOIN (
	SELECT DISTINCT ON (id_user) id_user, topup_value AS min_topup  
	/* change t2.topup_value to avoid the same name t1.topup_value */ 
	FROM topups_v2 ORDER BY id_user, topup_date) t2 
ON t1.id_user = t2.id_user

LEFT JOIN (
	SELECT id_user, topup_date AS previous_qual_topup_dt 
	FROM topups_v2 WHERE topup_value >= 20 ORDER BY id_user, topup_date DESC) t3
ON t1.id_user = t3.id_user 
AND t1.topup_date > t3.previous_qual_topup_dt)

SELECT id_user, topup_date, topup_value, 
		prv_topup_dt, days_since, promo_ind, 
		MAX(previous_qual_topup_dt) AS previous_qual_topup_dt, to_1st_ratio FROM tem_task5_v2  
GROUP BY id_user, topup_date, topup_value, 
		prv_topup_dt, days_since, promo_ind, to_1st_ratio
ORDER BY id_user, topup_date DESC;'''

curr.execute(sql9)

In [14]:
# view table task 5 using pandas
sql10 = '''SELECT * FROM task5_v2;'''
curr.execute(sql10)
pd.read_sql(sql10, conn)
#for table in curr.fetchall():
    #print(table)

,id_user,topup_date,topup_value,prv_topup_dt,days_since,promo_ind,previous_qual_topup_dt,to_1st_ratio
0,1,2017-06-20,15,2017-05-22,29.0,N,2017-04-17,0.6
1,1,2017-05-22,10,2017-04-17,35.0,N,2017-04-17,0.4
2,1,2017-04-17,20,2017-03-20,28.0,Y,2017-03-20,0.8
3,1,2017-03-20,20,2017-02-20,28.0,Y,2017-01-28,0.8
4,1,2017-02-20,15,2017-01-28,23.0,N,2017-01-28,0.6
...,...,...,...,...,...,...,...,...
63,5,2016-12-11,5,2016-12-07,4.0,N,2016-11-16,0.3
64,5,2016-12-07,10,2016-11-17,20.0,N,2016-11-16,0.5
65,5,2016-11-17,15,2016-11-16,1.0,N,2016-11-16,0.8
66,5,2016-11-16,20,None,NaN,Y,None,1.0


In [15]:
# task 6
sql11 = '''CREATE TABLE tem1_task6_v2 AS SELECT id_user, topup_date AS promo_start, topup_date + 28 AS promo_end 
FROM task5_v2 WHERE topup_value >= 20 ORDER BY id_user, topup_date;

SELECT * FROM tem1_task6_v2;

/* save to the final table: task6 */
CREATE TABLE task6_v2 AS WITH tem2_task6_v2 AS (SELECT t1.id_user, t1.promo_start, t2.promo_end FROM tem1_task6_v2 AS t1
 
LEFT JOIN tem1_task6_v2 AS t2  
ON t1.promo_end <= t2.promo_end
AND t1.id_user = t2.id_user
 
INNER JOIN tem1_task6_v2 AS t3      
ON t1.id_user = t3.id_user 
GROUP BY t1.id_user, t1.promo_start, t2.promo_end 
HAVING COUNT(CASE WHEN (t3.promo_start < t1.promo_start AND t1.promo_start <= t3.promo_end)  
                        OR (t3.promo_start <= t2.promo_end AND t2.promo_end < t3.promo_end) THEN 1 END) = 0)

SELECT id_user, promo_start, MIN(promo_end) AS promo_end FROM tem2_task6_v2  
GROUP BY id_user, promo_start;'''
curr.execute(sql11)

In [16]:
# view table task 6 using pandas
sql12 = '''SELECT * FROM task6_v2;'''
curr.execute(sql12)
pd.read_sql(sql12, conn)

,id_user,promo_start,promo_end
0,1,2016-11-07,2017-01-02
1,1,2017-01-03,2017-02-25
2,1,2017-03-20,2017-05-15
3,2,2017-01-15,2017-02-12
4,3,2016-12-19,2017-01-16
5,3,2017-02-01,2017-03-01
6,3,2017-03-15,2017-05-03
7,3,2017-05-11,2017-06-08
8,4,2016-12-20,2017-04-12
9,4,2017-05-30,2017-07-15


In [17]:
# close connection
conn.close()